# SYNERGxDB

https://www.synergxdb.ca/documentation/

**API endpoints**
* Samples ---> 1975 
* Compounds --
* Datasets --> son solo 9
* Drug Combinations --> complicado 
* Biomarkers
* Pharmacogenomic

Hay algunas quese pueden descargar pero haciendo webscrapping se añade el id

**datasets**

In [1]:
import requests

url = "https://www.synergxdb.ca/api/datasets/"

try:
    response = requests.get(url)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Error: {e}")

In [3]:
import pandas as pd 
# Load JSON response
datasets_info = response.json()
    
# Convert to DataFrame
df_datasets = pd.DataFrame(datasets_info)

In [4]:
df_datasets

,idSource,name,no_samples,no_drugs,pmID,author,combo
0,2,NCI-ALMANAC,60,101,28446463,Holbeck et al,3-by-3 or 5-by-3
1,1,MERCK,39,38,26983881,O'Neil et al,4-by-4
2,7,MIT-MELANOMA,36,108,26461489,Friedman et al,2-by-2
3,10,VISAGE,34,2,31302152,Patterson et al,10-by-6
4,9,DECREASE,13,33,None,Ianevski et al,8-by-8
5,5,YALE-TNBC,6,130,27872098,Wali et al,1-by-5
6,4,YALE-PDAC,4,41,28292938,Langdon et al,3-by-3
7,8,STANFORD,1,1818,28601558,Forcina et al,1-by-1
8,6,CLOUD,1,55,28530711,Licciardello et al,1-by-1


In [6]:
df_datasets.to_csv(r"C:\Users\Usuario\Desktop\TFM\datos\synergy\datasets.csv", index=False)


**cell lines**

In [7]:
import requests

url = "http://synergxdb.ca/api/cell_lines/"

try:
    response = requests.get(url)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Error: {e}")

In [8]:
# Load JSON response
datasets_info = response.json()
    
# Convert to DataFrame
df_cell_lines = pd.DataFrame(datasets_info)

In [9]:
df_cell_lines

,name,idSample,tissue,sex,age,disease,idCellosaurus,origin
0,K-562,1,blood,F,53.0,Chronic myelogenous leukemia,CVCL_0004,None
1,MOLT-4,2,blood,M,19.0,Adult T acute lymphoblastic leukemia,CVCL_0013,None
2,RPMI-8226,3,blood,M,61.0,Plasma cell myeloma,CVCL_0014,None
3,U-251MG,4,brain,M,75.0,Astrocytoma,CVCL_0021,None
4,A-549,5,lung,M,58.0,Lung adenocarcinoma,CVCL_0023,None
...,...,...,...,...,...,...,...,...
146,MDA-MB-453,209,breast,F,48.0,Breast carcinoma,CVCL_0418,Pleural effusion
147,NCI-H1299,210,lung,M,43.0,Large cell lung carcinoma,CVCL_0060,Lymph node
148,SW48,211,colorectal,F,83.0,Colon adenocarcinoma,CVCL_1724,None
149,U2OS,212,bone,F,15.0,Osteosarcoma,CVCL_0042,None


perfe porque tenemos info del diesease no hay que hacer webscrapping

In [10]:
df_cell_lines.to_csv(r"C:\Users\Usuario\Desktop\TFM\datos\synergy\cell_lines.csv", index=False)

**drugs**

In [11]:
import requests

url = "https://www.synergxdb.ca/api/drugs/"

try:
    response = requests.get(url)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Error: {e}")

In [12]:
# Load JSON response
datasets_info = response.json()
    
# Convert to DataFrame
df_drugs = pd.DataFrame(datasets_info)

In [62]:
df_drugs.nunique()

name             1975
atcCode           426
idDrugBank        770
idPubChem        1954
idDrug           1975
smiles           1936
inchikey         1955
dataset_names      94
dtype: int64

In [14]:
df_drugs.to_csv(r"C:\Users\Usuario\Desktop\TFM\datos\synergy\drugs.csv", index=False)

**Drug combinations**

In [19]:
import requests
import pandas as pd
from time import sleep

# API base URL
base_url = "https://www.synergxdb.ca/api/combos"

# Dataset information
datasets = {
    "MERCK": {"id": 1},
    "NCI-ALMANAC	": {"id": 2},
    "NO EXISTE": {"id": 3},
    "YALE-PDAC": {"id": 4},
    "NCI-ALMANAC": {"id": 5},
    "YALE-TNBC": {"id": 6},
    "MIT-MELANOMA": {"id": 7},
    "STANDFORD": {"id": 8},
    "DECREASE": {"id": 9}
}

per_page = 500  # Maximum per API documentation
all_data = []

def scrape_dataset(dataset_name, dataset_id):
    page = 1
    dataset_results = []
    previous_data = None
    
    while True:
        params = {
            "dataset": dataset_id,
            "page": page,
            "perPage": per_page
        }

        try:
            response = requests.get(base_url, params=params)
            
            if response.status_code == 404:
                print(f"[404] Dataset {dataset_name} (ID: {dataset_id}) not found.")
                break
            
            response.raise_for_status()
            data = response.json()
            
            # Stop if data repeats or no new data is received
            if not data or data == previous_data:
                print(f"Stopping at page {page} for dataset {dataset_name} (No new data).")
                break

            # Tag dataset for reference and store results
            for item in data:
                item["dataset"] = dataset_name  
            dataset_results.extend(data)
            previous_data = data
            
            print(f"Dataset {dataset_name} | Page {page} | Records: {len(data)}")
            
            # If less than per_page records, it's the last page
            if len(data) < per_page:
                print(f"Last page detected for {dataset_name} at page {page}.")
                break

            page += 1

        except requests.RequestException as e:
            print(f"Error scraping Dataset {dataset_name} | Page {page}: {e}")
            break

        sleep(1)  # Avoid hitting API rate limits
    
    print(f"Total records fetched for {dataset_name}: {len(dataset_results)}\n")
    return dataset_results

# Scraping all datasets
for dataset_name, info in datasets.items():
    print(f"Starting to scrape dataset: {dataset_name}")
    data = scrape_dataset(dataset_name, info["id"])
    all_data.extend(data)


Starting to scrape dataset: CLOUD
Dataset CLOUD | Page 1 | Records: 500
Dataset CLOUD | Page 2 | Records: 500
Dataset CLOUD | Page 3 | Records: 500
Dataset CLOUD | Page 4 | Records: 500
Dataset CLOUD | Page 5 | Records: 500
Dataset CLOUD | Page 6 | Records: 500
Dataset CLOUD | Page 7 | Records: 500
Dataset CLOUD | Page 8 | Records: 500
Dataset CLOUD | Page 9 | Records: 500
Dataset CLOUD | Page 10 | Records: 500
Dataset CLOUD | Page 11 | Records: 500
Dataset CLOUD | Page 12 | Records: 500
Dataset CLOUD | Page 13 | Records: 500
Dataset CLOUD | Page 14 | Records: 500
Dataset CLOUD | Page 15 | Records: 500
Dataset CLOUD | Page 16 | Records: 500
Dataset CLOUD | Page 17 | Records: 500
Dataset CLOUD | Page 18 | Records: 500
Dataset CLOUD | Page 19 | Records: 500
Dataset CLOUD | Page 20 | Records: 500
Dataset CLOUD | Page 21 | Records: 500
Dataset CLOUD | Page 22 | Records: 500
Dataset CLOUD | Page 23 | Records: 500
Dataset CLOUD | Page 24 | Records: 500
Dataset CLOUD | Page 25 | Records: 500


In [21]:
# Convert to DataFrame
df = pd.DataFrame(all_data)

# Display final stats
print(f"Total combinations fetched: {len(df)}")


Total combinations fetched: 531945


In [22]:
df.to_csv(r"C:\Users\Usuario\Desktop\TFM\datos\synergy\combinations_v1.csv", index=False)


In [16]:
df_combinations_v1 = pd.read_csv(r"C:\Users\Usuario\Desktop\TFM\datos\synergy\combinations_v1.csv")

In [ ]:
# Dataset information
datasets = {
    "MERCK": {"id": 1},
    "NCI-ALMANAC	": {"id": 2},
    "NO EXISTE": {"id": 3},
    "YALE-PDAC": {"id": 4},
    "NCI-ALMANAC": {"id": 5},
    "YALE-TNBC": {"id": 6},
    "MIT-MELANOMA": {"id": 7},
    "STANDFORD": {"id": 8},
    "DECREASE": {"id": 9}
}

In [19]:
df_combinations_v1 = df_combinations_v1.drop(columns=['dataset'])

In [25]:
df_combinations_v1.columns #tiene la informacion incluida del cell line y del drug 

Index(['comboId', 'idSample', 'bliss', 'loewe', 'hsa', 'zip', 'comboscore',
       'sourceName', 'sampleName', 'drugNameA', 'drugNameB', 'tissue',
       'idSource', 'idDrugA', 'idDrugB', 'idCellosaurus', 'sex', 'age',
       'disease', 'origin', 'atCodeDrugA', 'idDrugBankA', 'idPubChemDrugA',
       'atCodeDrugB', 'idDrugBankB', 'idPubChemDrugB', 'smilesDrugA',
       'inchikeyDrugA', 'smilesDrugB', 'inchikeyDrugB'],
      dtype='object')

In [30]:
# Display specific columns from the DataFra
df_combinations_v1[['comboId','sampleName', 'idDrugA', 'idDrugB', 'drugNameA', 'drugNameB','zip','disease']]


,comboId,sampleName,idDrugA,idDrugB,drugNameA,drugNameB,zip,disease
0,12543,A2058,52,1,Lapatinib,MK-8776,23.8255,Amelanotic melanoma
1,3218,HCT 116,52,34,Lapatinib,Erlotinib,23.7600,Colon carcinoma
2,18087,EFM-192B,103,1,Vinblastine,MK-8776,18.3098,Breast carcinoma
3,3164,HCT 116,41,106,Fluorouracil,Vorinostat,17.7934,Colon carcinoma
4,11963,A-427,52,34,Lapatinib,Erlotinib,17.7400,Lung adenocarcinoma
...,...,...,...,...,...,...,...,...
531940,592093,CAL-85-1,128,131,Navitoclax,Trametinib,-1.1144,Breast carcinoma
531941,592249,MDA-MB-436,128,131,Navitoclax,Trametinib,-1.1209,"Invasive ductal carcinoma, not otherwise speci..."
531942,592247,MDA-MB-436,1125,131,Iniparib,Trametinib,-1.1363,"Invasive ductal carcinoma, not otherwise speci..."
531943,592257,MDA-MB-436,217,131,Entinostat,Trametinib,-1.2432,"Invasive ductal carcinoma, not otherwise speci..."


ejemplo para ver los duplicados

In [34]:
df_combinations_v1[
    (df_combinations_v1['drugNameA'] == 'Navitoclax') & (df_combinations_v1['drugNameB'] == 'Trametinib')
][['comboId','sampleName', 'drugNameA', 'drugNameB','zip','disease','origin']]


,comboId,sampleName,drugNameA,drugNameB,zip,disease,origin
531736,592195,HCC1599,Navitoclax,Trametinib,3.3157,Ductal breast carcinoma,NaN
531778,592169,DU4475,Navitoclax,Trametinib,-0.6840,Breast carcinoma,Skin nodule
531837,592277,MDA-MB-231,Navitoclax,Trametinib,-0.8041,Breast adenocarcinoma,Pleural effusion
531902,592139,CAL51,Navitoclax,Trametinib,-0.9534,Breast carcinoma,Pleural effusion
531905,592109,CAL-148,Navitoclax,Trametinib,-0.9663,Breast carcinoma,Pleural effusion
531908,592207,MCF-10A,Navitoclax,Trametinib,-0.9686,NaN,NaN
531916,592219,MFM-223,Navitoclax,Trametinib,-0.9920,Breast carcinoma,Pleural effusion
531933,592261,BT-549,Navitoclax,Trametinib,-1.0313,Invasive ductal carcinoma,NaN
531940,592093,CAL-85-1,Navitoclax,Trametinib,-1.1144,Breast carcinoma,NaN
531941,592249,MDA-MB-436,Navitoclax,Trametinib,-1.1209,"Invasive ductal carcinoma, not otherwise speci...",Pleural effusion


In [36]:
print(df_combinations_v1.shape[0])
df_combinations_v1.nunique()

531945


comboId           530561
idSample             133
bliss              28230
loewe              17701
hsa                24118
zip                20286
comboscore          1150
sourceName             8
sampleName           133
drugNameA           1937
drugNameB            325
tissue                13
idSource               8
idDrugA             1937
idDrugB              325
idCellosaurus        125
sex                    2
age                   50
disease               45
origin                17
atCodeDrugA          417
idDrugBankA          755
idPubChemDrugA      1924
atCodeDrugB          160
idDrugBankB          203
idPubChemDrugB       304
smilesDrugA         1907
inchikeyDrugA       1925
smilesDrugB          303
inchikeyDrugB        305
dtype: int64

podemos ver que hay comboId que no son unicos, eso los vamos a quitar. Tambien quitar por cell_line_druga_drugb

In [43]:
print(df_combinations_v1['disease'].unique())
df_combinations_v1['tissue'].unique()


['Amelanotic melanoma' 'Colon carcinoma' 'Breast carcinoma'
 'Lung adenocarcinoma' 'Ovarian serous cystadenocarcinoma'
 'Ovarian endometrioid adenocarcinoma'
 'Minimally invasive lung adenocarcinoma' 'Large cell lung carcinoma'
 'Prostate carcinoma' 'Squamous cell lung carcinoma'
 'Rectal adenocarcinoma' 'High grade ovarian serous adenocarcinoma'
 'Rectosigmoid adenocarcinoma'
 'Invasive ductal carcinoma, not otherwise specified' 'Ovarian carcinoma'
 'Ovarian clear cell adenocarcinoma' 'Ovarian adenocarcinoma'
 'Ovarian mixed germ cell tumor' 'Cutaneous melanoma' 'Melanoma'
 'Colon adenocarcinoma' 'Pleural biphasic mesothelioma'
 'Breast adenocarcinoma' 'Childhood T acute lymphoblastic leukemia'
 'Papillary renal cell carcinoma' 'Adult T acute lymphoblastic leukemia'
 'Lung adenocarcinoma ' 'Gliosarcoma' 'Adult acute myeloid leukemia'
 'Plasma cell myeloma' 'Clear cell renal cell carcinoma ' 'Astrocytoma'
 'Renal cell carcinoma' 'Invasive ductal carcinoma'
 'Anaplastic large cell lymph

array(['skin', 'colorectal', 'breast', 'lung', 'ovary', 'prostate',
       'mesothelioma', 'blood', 'kidney', 'brain', 'pancreas', 'liver',
       'cervix'], dtype=object)

filtramos por cell line drug a drug b 

In [65]:
import pandas as pd

# Example DataFrame
# Assuming df_combinations_v1 is already defined in your environment

# Create a new column by combining 'idDrugA', 'idDrugB', and 'idSample'
df_combinations_v1['combined_key'] = (
    df_combinations_v1['idDrugA'].astype(str) + "_" +
    df_combinations_v1['idDrugB'].astype(str) + "_" +
    df_combinations_v1['idSample'].astype(str)
)

# Filter unique values of the combined key
unique_combinations_df = df_combinations_v1.drop_duplicates(subset=['combined_key'])
unique_combinations_df = unique_combinations_df.drop(columns=['combined_key'])


In [66]:
unique_combinations_df

,comboId,idSample,bliss,loewe,hsa,zip,comboscore,sourceName,sampleName,drugNameA,...,atCodeDrugA,idDrugBankA,idPubChemDrugA,atCodeDrugB,idDrugBankB,idPubChemDrugB,smilesDrugA,inchikeyDrugA,smilesDrugB,inchikeyDrugB
0,12543,45,-1.2580,-0.2988,-0.3583,23.8255,NaN,MERCK,A2058,Lapatinib,...,L01XE07,DB01259,208908.0,NaN,DB11899,16224745.0,CS(=O)(=O)CCNCC1=CC=C(O1)C2=CC3=C(C=C2)N=CN=C3...,BCFGMOOMADDAQU-UHFFFAOYSA-N,CN1C=C(C=N1)C2=C3N=C(C(=C(N3N=C2)N)Br)C4CCCNC4,GMIZZEXBPRLVIV-UHFFFAOYSA-N
1,3218,20,-1.1097,-0.1197,-0.1197,23.7600,NaN,MERCK,HCT 116,Lapatinib,...,L01XE07,DB01259,208908.0,L01XE03,DB00530,176870.0,CS(=O)(=O)CCNCC1=CC=C(O1)C2=CC3=C(C=C2)N=CN=C3...,BCFGMOOMADDAQU-UHFFFAOYSA-N,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,AAKJLRGGTJKAMG-UHFFFAOYSA-N
2,18087,81,-0.2518,0.5204,-0.2360,18.3098,NaN,MERCK,EFM-192B,Vinblastine,...,L01CA01,DB00570,13342.0,NaN,DB11899,16224745.0,CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...,JXLYSJRDGCGARV-CFWMRBGOSA-N,CN1C=C(C=N1)C2=C3N=C(C(=C(N3N=C2)N)Br)C4CCCNC4,GMIZZEXBPRLVIV-UHFFFAOYSA-N
3,3164,20,-1.1739,-0.1872,-0.4840,17.7934,NaN,MERCK,HCT 116,Fluorouracil,...,L01BC02;L01BC52,DB00544,3385.0,L01XX38,DB02546,5311.0,C1=C(C(=O)NC(=O)N1)F,GHASVSINZRGABV-UHFFFAOYSA-N,C1=CC=C(C=C1)NC(=O)CCCCCCC(=O)NO,WAEXFXRVDQXREF-UHFFFAOYSA-N
4,11963,43,-1.0380,-0.1204,-0.2398,17.7400,NaN,MERCK,A-427,Lapatinib,...,L01XE07,DB01259,208908.0,L01XE03,DB00530,176870.0,CS(=O)(=O)CCNCC1=CC=C(O1)C2=CC3=C(C=C2)N=CN=C3...,BCFGMOOMADDAQU-UHFFFAOYSA-N,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,AAKJLRGGTJKAMG-UHFFFAOYSA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531940,592093,187,-1.1680,-0.1928,-0.2052,-1.1144,NaN,DECREASE,CAL-85-1,Navitoclax,...,NaN,DB12340,24978538.0,L01XE25,DB08911,11707110.0,CC1(CCC(=C(C1)CN2CCN(CC2)C3=CC=C(C=C3)C(=O)NS(...,JLYAXFNOILIKPP-KXQOOQHDSA-N,CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N(...,LIRYPHYGHXZJBZ-UHFFFAOYSA-N
531941,592249,40,-1.2763,-0.1770,-0.3201,-1.1209,NaN,DECREASE,MDA-MB-436,Navitoclax,...,NaN,DB12340,24978538.0,L01XE25,DB08911,11707110.0,CC1(CCC(=C(C1)CN2CCN(CC2)C3=CC=C(C=C3)C(=O)NS(...,JLYAXFNOILIKPP-KXQOOQHDSA-N,CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N(...,LIRYPHYGHXZJBZ-UHFFFAOYSA-N
531942,592247,40,-1.2557,-0.1758,-0.2597,-1.1363,NaN,DECREASE,MDA-MB-436,Iniparib,...,NaN,NaN,9796068.0,L01XE25,DB08911,11707110.0,C1=CC(=C(C=C1C(=O)N)[N+](=O)[O-])I,MDOJTZQKHMAPBK-UHFFFAOYSA-N,CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N(...,LIRYPHYGHXZJBZ-UHFFFAOYSA-N
531943,592257,40,-1.3453,-0.0859,-0.2181,-1.2432,NaN,DECREASE,MDA-MB-436,Entinostat,...,NaN,NaN,4261.0,L01XE25,DB08911,11707110.0,C1=CC=C(C(=C1)N)NC(=O)C2=CC=C(C=C2)CNC(=O)OCC3...,INVTYAOGFAGBOE-UHFFFAOYSA-N,CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N(...,LIRYPHYGHXZJBZ-UHFFFAOYSA-N


In [67]:
unique_combinations_df.nunique()

comboId           530561
idSample             133
bliss              28186
loewe              17681
hsa                24067
zip                20278
comboscore          1150
sourceName             8
sampleName           133
drugNameA           1937
drugNameB            325
tissue                13
idSource               8
idDrugA             1937
idDrugB              325
idCellosaurus        125
sex                    2
age                   50
disease               45
origin                17
atCodeDrugA          417
idDrugBankA          755
idPubChemDrugA      1924
atCodeDrugB          160
idDrugBankB          203
idPubChemDrugB       304
smilesDrugA         1907
inchikeyDrugA       1925
smilesDrugB          303
inchikeyDrugB        305
dtype: int64

530561 combinaciones, 133 lineas celulares, 45 enfermedades, 1974 diferentes drugs

In [69]:
unique_drugs = pd.unique(pd.concat([df_combinations_v1['idDrugA'], df_combinations_v1['idDrugB']]))
len(unique_drugs)

1974

**Sistema nervioso central**

Los que tienen que ver con el sistema nervioso central: Glioblastoma Gliosarcoma Astrocytoma 

In [44]:
# List of CNS-related diseases
cns_diseases = ['Glioblastoma', 'Gliosarcoma', 'Astrocytoma']

# Filter the DataFrame based on the disease column
filtered_df = df_combinations_v1[df_combinations_v1['disease'].isin(cns_diseases)]

filtered_df
filtered_df[['comboId','sampleName', 'idDrugA', 'idDrugB', 'drugNameA', 'drugNameB','zip','disease','tissue']]


,comboId,sampleName,idDrugA,idDrugB,drugNameA,drugNameB,zip,disease,tissue
22725,319004,SF539,39,286,Floxuridine,Pazopanib,21.4159,Gliosarcoma,brain
22756,166494,SNB-19,63,58,Methotrexate,Megestrol Acetate,21.2754,Astrocytoma,brain
22761,311359,SF268,58,67,Megestrol Acetate,Mitoxantrone,21.2424,Astrocytoma,brain
22775,164892,SNB-19,32,254,Docetaxel,Vismodegib,21.1565,Astrocytoma,brain
22818,319552,SF539,51,8,Ixabepilone,Azacitidine,15.9849,Gliosarcoma,brain
...,...,...,...,...,...,...,...,...,...
531730,591009,T98G,1702,92,Saracatinib,Temozolomide,-1.3536,Glioblastoma,brain
531731,590460,T98G,1177,92,Ceritinib,Temozolomide,-1.3548,Glioblastoma,brain
531732,590988,T98G,1684,92,SGI-1776,Temozolomide,-1.3614,Glioblastoma,brain
531733,590899,T98G,1598,92,RG-2833,Temozolomide,-1.4191,Glioblastoma,brain


In [52]:
print(filtered_df.shape[0])
filtered_df.nunique()

29954


comboId           29954
idSample              7
bliss              5124
loewe              5773
hsa                5934
zip                6795
comboscore          650
sourceName            2
sampleName            7
drugNameA          1856
drugNameB           102
tissue                1
idSource              2
idDrugA            1856
idDrugB             102
idCellosaurus         7
sex                   2
age                   5
disease               3
origin                0
atCodeDrugA         396
idDrugBankA         729
idPubChemDrugA     1856
atCodeDrugB          96
idDrugBankB         102
idPubChemDrugB      102
smilesDrugA        1841
inchikeyDrugA      1856
smilesDrugB         101
inchikeyDrugB       102
dtype: int64

es igual, solo hay estas 3 enfermedades

In [73]:
unique_drugs = pd.unique(pd.concat([filtered_df['idDrugA'], filtered_df['idDrugB']]))
len(unique_drugs)

1856

29954 combinaciones, 7 lineas celulares, 3 enfermedades, 1856 drugs

In [70]:
import pandas as pd

# Example DataFrame
# Assuming df_combinations_v1 is already defined in your environment

# Create a new column by combining 'idDrugA', 'idDrugB', and 'idSample'
filtered_df['combined_key'] = (
    filtered_df['idDrugA'].astype(str) + "_" +
    filtered_df['idDrugB'].astype(str) + "_" +
    filtered_df['idSample'].astype(str)
)

# Filter unique values of the combined key
unique_combinations_df = filtered_df.drop_duplicates(subset=['combined_key'])
unique_combinations_df = filtered_df.drop(columns=['combined_key'])


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12504\3899842488.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['combined_key'] = (


In [72]:
unique_combinations_df.shape[0] ##es igual no habia nigun duplicado 

29954